Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [8]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))#x is 16*28*28*1 so 16 images per batch
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1)) #5*5*1*16 #16- 5*5 convs with strid 2
  layer1_biases = tf.Variable(tf.zeros([depth])) 
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))#16- 5*5 convs with stride 2
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))# 784*64 # two convs with stride 2 make the dimention a quarter.
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1)) #64*10
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')#Must have strides[0] = strides[3] = 1. For the most common case of the same horizontal and vertices strides, strides = [1, stride, stride, 1].
    hidden = tf.nn.relu(conv + layer1_biases) #hidden size 16*14*14*16
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)# hidden size is 16*7*7*16
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])# hidden size will be 16*784
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases) # now 16*64
    return tf.matmul(hidden, layer4_weights) + layer4_biases # returns 16*10
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [9]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.203400
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.911157
Minibatch accuracy: 25.0%
Validation accuracy: 48.6%
Minibatch loss at step 100: 1.021933
Minibatch accuracy: 62.5%
Validation accuracy: 66.8%
Minibatch loss at step 150: 0.506469
Minibatch accuracy: 81.2%
Validation accuracy: 74.7%
Minibatch loss at step 200: 0.974376
Minibatch accuracy: 75.0%
Validation accuracy: 76.2%
Minibatch loss at step 250: 1.168719
Minibatch accuracy: 68.8%
Validation accuracy: 77.9%
Minibatch loss at step 300: 0.412626
Minibatch accuracy: 81.2%
Validation accuracy: 79.0%
Minibatch loss at step 350: 0.569507
Minibatch accuracy: 93.8%
Validation accuracy: 78.3%
Minibatch loss at step 400: 0.234789
Minibatch accuracy: 93.8%
Validation accuracy: 79.5%
Minibatch loss at step 450: 0.794356
Minibatch accuracy: 81.2%
Validation accuracy: 79.0%
Minibatch loss at step 500: 0.635681
Minibatch accuracy: 87.5%
Validation accuracy: 81.0%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [11]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))#x is 16*28*28*1 so 16 images per batch
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1)) #5*5*1*16 #16- 5*5 convs with strid 2
  layer1_biases = tf.Variable(tf.zeros([depth])) 
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))#16- 5*5 convs with stride 2
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))# 784*64 # two convs with stride 2 make the dimention a quarter.
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1)) #64*10
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')#Must have strides[0] = strides[3] = 1. For the most common case of the same horizontal and vertices strides, strides = [1, stride, stride, 1].
    hidden = tf.nn.relu(conv + layer1_biases) #hidden size 16*28*28*16
    pool = tf.nn.max_pool(hidden,
                          ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1],
                          padding='SAME') #pool size 16*14*14*16
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)# hidden size is 16*14*14*16
    pool = tf.nn.max_pool(hidden,
                          ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1],
                          padding='SAME') #pool size 16*7*7*16
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])# hidden size will be 16*784
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases) # now 16*64
    return tf.matmul(hidden, layer4_weights) + layer4_biases # returns 16*10
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [12]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.951349
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.263766
Minibatch accuracy: 6.2%
Validation accuracy: 22.6%
Minibatch loss at step 100: 1.638675
Minibatch accuracy: 31.2%
Validation accuracy: 34.2%
Minibatch loss at step 150: 0.601856
Minibatch accuracy: 87.5%
Validation accuracy: 72.3%
Minibatch loss at step 200: 1.169322
Minibatch accuracy: 68.8%
Validation accuracy: 76.6%
Minibatch loss at step 250: 1.119571
Minibatch accuracy: 62.5%
Validation accuracy: 77.8%
Minibatch loss at step 300: 0.374689
Minibatch accuracy: 87.5%
Validation accuracy: 80.2%
Minibatch loss at step 350: 0.648457
Minibatch accuracy: 87.5%
Validation accuracy: 79.2%
Minibatch loss at step 400: 0.213703
Minibatch accuracy: 93.8%
Validation accuracy: 80.3%
Minibatch loss at step 450: 0.972452
Minibatch accuracy: 81.2%
Validation accuracy: 80.9%
Minibatch loss at step 500: 0.566014
Minibatch accuracy: 87.5%
Validation accuracy: 81.7%
Mi

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [44]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

beta = 1e-4
graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))#x is 16*28*28*1 so 16 images per batch
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1)) #5*5*1*16 #16- 5*5 convs with strid 2
  layer1_biases = tf.Variable(tf.zeros([depth])) 
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))#16- 5*5 convs with stride 2
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))# 784*64 # two convs with stride 2 make the dimention a quarter.
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1)) #64*10
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, train=False):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')#Must have strides[0] = strides[3] = 1. For the most common case of the same horizontal and vertices strides, strides = [1, stride, stride, 1].
    hidden = tf.nn.relu(conv + layer1_biases) #hidden size 16*28*28*16
    pool = tf.nn.max_pool(hidden,
                          ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1],
                          padding='SAME') #pool size 16*14*14*16
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)# hidden size is 16*14*14*16
    pool = tf.nn.max_pool(hidden,
                          ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1],
                          padding='SAME') #pool size 16*7*7*16
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])# hidden size will be 16*784
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases) # now 16*64
    #if train:
      #hidden = tf.nn.dropout(hidden, 0.5)
    return tf.matmul(hidden, layer4_weights) + layer4_biases # returns 16*10
  
  # Training computation.
  logits = model(tf_train_dataset,True)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  #loss += (beta/2)*tf.nn.l2_loss(layer1_weights)+ (beta/2)*tf.nn.l2_loss(layer2_weights)+(beta/2)*tf.nn.l2_loss(layer3_weights)+(beta/2)*tf.nn.l2_loss(layer4_weights)
      
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  decay_steps = 100 # decay every 100 steps
  decay_rate = 0.95 # the exponetial base
  learning_rate = tf.train.exponential_decay(0.1, global_step, decay_steps, decay_rate)
  optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [45]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
#92.3 with no dropout and regularization

Initialized
Minibatch loss at step 0: 2.980749
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.101794
Minibatch accuracy: 50.0%
Validation accuracy: 67.1%
Minibatch loss at step 100: 0.676147
Minibatch accuracy: 75.0%
Validation accuracy: 77.3%
Minibatch loss at step 150: 0.302132
Minibatch accuracy: 87.5%
Validation accuracy: 79.3%
Minibatch loss at step 200: 0.810153
Minibatch accuracy: 81.2%
Validation accuracy: 80.6%
Minibatch loss at step 250: 1.347921
Minibatch accuracy: 68.8%
Validation accuracy: 81.5%
Minibatch loss at step 300: 0.336636
Minibatch accuracy: 87.5%
Validation accuracy: 82.6%
Minibatch loss at step 350: 0.375231
Minibatch accuracy: 93.8%
Validation accuracy: 81.3%
Minibatch loss at step 400: 0.203974
Minibatch accuracy: 100.0%
Validation accuracy: 82.1%
Minibatch loss at step 450: 1.249485
Minibatch accuracy: 75.0%
Validation accuracy: 81.9%
Minibatch loss at step 500: 0.377586
Minibatch accuracy: 93.8%
Validation accuracy: 83.8%
M